In [2]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross, MeanReversionLongOnly
from src.utils import load_data

warnings.filterwarnings('ignore')
#pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
ticker = "^SPX"
period = "max"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [4]:
data.dropna()
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-10-21 00:00:00-04:00,5857.819824,5866.919922,5824.790039,5853.979980,3407010000,0.0,0.0
2024-10-22 00:00:00-04:00,5832.700195,5863.040039,5821.169922,5851.200195,3342080000,0.0,0.0
2024-10-23 00:00:00-04:00,5834.500000,5834.850098,5762.410156,5797.419922,3532650000,0.0,0.0


In [5]:
strategy = SmaCross
strategy.n1 = 60
strategy.n2 = 200
bt_sma = Backtest(data["1998":], SmaCross, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_sma.run()
stats

Start                     1998-01-02 00:00...
End                       2024-10-25 00:00...
Duration                   9792 days 23:00:00
Exposure Time [%]                   69.531713
Equity Final [$]                 54273.466353
Equity Peak [$]                   54766.75688
Return [%]                         442.734664
Buy & Hold Return [%]              495.680203
Return (Ann.) [%]                    6.520386
Volatility (Ann.) [%]               13.358446
Sharpe Ratio                          0.48811
Sortino Ratio                        0.727238
Calmar Ratio                         0.201109
Max. Drawdown [%]                  -32.422081
Avg. Drawdown [%]                   -1.937662
Max. Drawdown Duration     1257 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                                   11
Win Rate [%]                        90.909091
Best Trade [%]                      43.964921
Worst Trade [%]                    -13.878987
Avg. Trade [%]                    

In [6]:
bt_sma.plot()

Row(id='1450', ...)

In [8]:
%%time
stats = bt_sma.optimize(
    n1=range(20, 70, 10),
    n2=range(100, 250, 20),
    maximize='Equity Final [$]',
    constraint=lambda param: param.n1 < param.n2,
)
stats

  0%|          | 0/14 [00:00<?, ?it/s]

CPU times: user 7.27 s, sys: 44.1 ms, total: 7.32 s
Wall time: 7.31 s


Start                     1998-01-02 00:00...
End                       2024-10-25 00:00...
Duration                   9792 days 23:00:00
Exposure Time [%]                   69.531713
Equity Final [$]                 54273.466353
Equity Peak [$]                   54766.75688
Return [%]                         442.734664
Buy & Hold Return [%]              495.680203
Return (Ann.) [%]                    6.520386
Volatility (Ann.) [%]               13.358446
Sharpe Ratio                          0.48811
Sortino Ratio                        0.727238
Calmar Ratio                         0.201109
Max. Drawdown [%]                  -32.422081
Avg. Drawdown [%]                   -1.937662
Max. Drawdown Duration     1257 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                                   11
Win Rate [%]                        90.909091
Best Trade [%]                      43.964921
Worst Trade [%]                    -13.878987
Avg. Trade [%]                    

In [9]:
stats._strategy

<Strategy SmaCross(n1=60,n2=200)>

In [10]:
bt_sma.plot()

Row(id='2182', ...)

In [11]:
%%time
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=False)
stats = bt_momentum.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

  0%|          | 0/9 [00:00<?, ?it/s]

CPU times: user 10.1 s, sys: 43 ms, total: 10.1 s
Wall time: 10.1 s


Start                     1927-12-30 00:00...
End                       2024-10-25 00:00...
Duration                  35363 days 23:00:00
Exposure Time [%]                    1.089549
Equity Final [$]                     5.140937
Equity Peak [$]                       10000.0
Return [%]                         -99.948591
Buy & Hold Return [%]            32788.562669
Return (Ann.) [%]                   -7.546546
Volatility (Ann.) [%]                5.084306
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.949791
Avg. Drawdown [%]                  -99.949791
Max. Drawdown Duration    35343 days 00:00:00
Avg. Drawdown Duration    35343 days 00:00:00
# Trades                                  222
Win Rate [%]                         1.801802
Best Trade [%]                        3.41795
Worst Trade [%]                    -10.477453
Avg. Trade [%]                    

In [10]:
bt_momentum.plot()

TypeError: Index.get_loc() got an unexpected keyword argument 'method'

In [ ]:
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=True)
stats = bt_momentum.run()
stats